In [ ]:
# Jinko specifics imports & initialization
# Please fold this section and do not edit it
import sys

sys.path.insert(0, "../lib")
import jinko_helpers as jinko

# Connect to Jinko (see README.md for more options)
jinko.initialize()

In [ ]:
# Cookbook specifics imports
import json

## Select calibration of interest

In [ ]:
"""
An optional folder location to store debug files
"""

folder = "."
# Set this to True to write debugging files
write_files = False

""" 
Calibration short id can be retrieved from the URL of your calibration in Jinko, pattern is `https://jinko.ai/<calibration_short_id>`
"""
calibration_short_id = "ca-6f5g-NiYH"
# Choose a spefic revision. By default we return the last version
revision = 18
# Choose a specific label. By default we return the last version
label = None
response = jinko.get_project_item_new(
    sid=calibration_short_id, revision=revision, label=label
)
calibration_core_item_id, calibration_snapshot_id = (
    response["coreId"]["id"],
    response["coreId"]["snapshotId"],
)

# # Uncomment the following if you want to use the latest completed or stopped version
# response = jinko.get_latest_calib_with_status(shortId=calibration_short_id, statuses=["completed", "stopped"])
# calibration_core_item_id, calibration_snapshot_id = response["coreItemId"], response["snapshotId"]

print(
    f"Picked Calibration with coreItemId: {calibration_core_item_id}, snapshotId: {calibration_snapshot_id}"
)

status = jinko.make_request(
    f"/core/v2/calibration_manager/calibration/{calibration_core_item_id}/snapshots/{calibration_snapshot_id}/status",
    method="GET",
).text
print(f"It has status: {status}")

## Getting the calibration summary

In [ ]:
summary_response = jinko.make_request(
    f"/core/v2/calibration_manager/calibration/{calibration_core_item_id}/snapshots/{calibration_snapshot_id}/results_summary",
    method="GET",
)

if write_files:
    with open(f"{folder}/calibration_summary.json", "w") as file:
        file.write(summary_response.text)

## Debug a failing calibration

In [ ]:
weighted_score_response = jinko.make_request(
    f"/core/v2/result_manager/calibration/scalar_result",
    method="POST",
    json={
        "calibId": {
            "coreItemId": calibration_core_item_id,
            "snapshotId": calibration_snapshot_id,
        },
        "patientId": None,
        "select": ["optimizationWeightedScore"],
    },
)
if write_files:
    with open(f"{folder}/optimizationWeightedScore.json", "w") as file:
        file.write(weighted_score_response.text)

try:
    weighted_score = next(
        r
        for r in json.loads(weighted_score_response.content)["outputs"]
        if "error" in r["res"][0] and r["res"][0]["error"] is not None
    )
except StopIteration:
    raise ("Could not find any error with the current calibration")
error_str = weighted_score["res"][0]["error"]
patient_id = weighted_score["indexes"]["patientNumber"]
print(
    f"Here is the first error found regarding optimizationWeightedScore:\n{error_str}"
)

"""
If the error message is well-formed we can automatically extract the faulty scalar name
"""
if "ScalarError" in error_str:
    scalar_id = error_str.lstrip('ScalarError "').rstrip('"')
    print(f"\nHere is the allegedly faulty scalar: {scalar_id}")

    scalar_response = jinko.make_request(
        f"/core/v2/result_manager/calibration/scalar_result",
        method="POST",
        json={
            "calibId": {
                "coreItemId": calibration_core_item_id,
                "snapshotId": calibration_snapshot_id,
            },
            "patientId": None,
            "select": [scalar_id],
        },
    )

    if write_files:
        with open(f"{folder}/individual_score.json", "w") as file:
            file.write(scalar_response.text)

    faulty_scalar = next(
        r
        for r in json.loads(scalar_response.content)["outputs"]
        if "error" in r["res"][0] and r["res"][0]["error"] is not None
    )
    error_str = faulty_scalar["res"][0]["error"]
    print(f"Here is the first error found regarding said scalar: {error_str}")
elif "MissingScalar" in error_str:
    scalar_id = error_str.lstrip('MissingScalar "').rstrip('"')
    print(f"\nThe following scalar is missing from the results: {scalar_id}")
    print(f"Something probably went wrong during solving, check out the ModelResult")

"""
Try and query the ModelResult for the faulty patient
"""

json_response = jinko.make_request(
    f"/core/v2/result_manager/calibration/model_result",
    method="POST",
    json={
        "calibId": {
            "coreItemId": calibration_core_item_id,
            "snapshotId": calibration_snapshot_id,
        },
        "patientId": patient_id,
        "select": ["Time"],
    },
).text
if write_files:
    model_result_filename = f"{folder}/individual_model_result.json"
    with open(model_result_filename, "w") as file:
        file.write(json_response)
    print(
        f"\nWrote the ModelResult associated with the faulty patient here: {model_result_filename}. Take a good look at it 👀"
    )